In [ ]:
import sys
from pprint import pprint
from typing import List, Any

import pybmd

from python_get_resolve import GetResolve

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
sub_folder = media_pool.GetRootFolder().GetSubFolderList()[0]

root_folder = media_pool.GetRootFolder()
for sub_folder in root_folder.GetSubFolderList():
    for clip in sub_folder.GetClipList():
        print(clip.GetName())


In [ ]:
from pybmd import Bmd

resolve = Bmd()
project = resolve.get_project_manager().get_current_project()
media_pool = project.get_media_pool()

clip_width = 4096

media_pool.create_empty_timeline("tmp")
timeline = project.get_current_timeline()
print(f"1_current_timeline: {timeline.get_name()}")

media_pool.create_empty_timeline("tmp1")
timeline = project.get_current_timeline()
print(f"2_current_timeline: {timeline.get_name()}")

media_pool.create_empty_timeline("tmp")
timeline_number = project.get_timeline_count()
for i in range(timeline_number):
    timeline = project.get_timeline_by_index(i + 1)
    # print(timeline.get_setting()["timelineResolutionWidth"])
    if timeline.get_setting()["timelineResolutionWidth"] == str(clip_width):
        while project.set_current_timeline(timeline):
            print("successful")
            break
timeline = project.get_current_timeline()
print(f"3_current_timeline: {timeline.get_name()}")


`createEmptyTimeline()` 在遇到名称与之前建立的时间线重复之时，会直接跳过，不采取什么动作。通常情况下 `createEmptyTimeline()` 之后会把 currentTimeline 设为新建的那条时间线，但是遇到重复的话。

What I need to do now: 找到目前新建的时间线当中，分辨率 width 与当前这个 clip 相符的那条时间线，把它设为 current timeline。

In [ ]:
from pybmd import Bmd
from pprint import pprint

resolve = Bmd()
project = resolve.get_project_manager().get_current_project()
media_pool = project.get_media_pool()
root_folder = media_pool.get_root_folder()

for sub_folder in root_folder.get_sub_folder_list():
    if sub_folder.get_name() == "Ronin_4D#2":
        for index, clip in enumerate(sub_folder.get_clip_list()):
            pprint(f"{index}_{clip.get_name()}")

In [8]:
from pybmd import Bmd
from pybmd.toolkits import change_timeline_resolution, get_all_timeline, get_timeline

resolve = Bmd()
project = resolve.get_project_manager().get_current_project()
media_pool = project.get_media_pool()
root_folder = media_pool.get_root_folder()
media_storage = resolve.get_media_stroage()

current_timeline = project.get_current_timeline()

change_timeline_resolution(current_timeline, 1920, 1080)
timeline = get_timeline(project, 'FX6#3_2048x856')
timeline.get_name()

media_storage.add_item_list_to_meida_pool()

'FX6#3_2048x856'

In [ ]:
from pybmd import Bmd

resolve = Bmd()
project = resolve.get_project_manager().get_current_project()
media_pool = project.get_media_pool()

# 拿到机位 bin 下所有 clip 的分辨率信息，create new empty timeline.
for index, sub_folder in enumerate(media_pool.get_root_folder().get_sub_folder_list()):

    # 排除 _Timeline Bin
    if sub_folder.get_name() == "_Timeline" or sub_folder.get_name() == "tmp":
        break

    # 拿到机位 bin 下所有 clip 的分辨率信息 assign to all_clips_resolution 这个 list.
    all_clips_resolution = []  # Camera 机位 bin 下所有 clip 的 resolution 信息
    for clips in sub_folder.get_clip_list():
        all_clips_resolution.append(clips.get_clip_property()["Resolution"])
    all_clips_resolution: list[str] = list(dict.fromkeys(all_clips_resolution))  # 移除 list 中的重复项
    print(f"{index + 1}. {sub_folder.get_name()}: {all_clips_resolution}")

    # 根据 all_clips_resolution 里的分辨率信息新建时间线
    for res in all_clips_resolution:
        timeline_name = f"{sub_folder.get_name()}_{res}"
        print(res)
        # media_pool.create_empty_timeline(timeline_name).set_setting("useCustomSettings", "1")

# 出现问题的原因是：

1. 同一个文件夹 Bin 当中有 2048x1080 的素材，也有 4096x2160 的素材。后者再除以 2 之后正好又等于前者。所以在先建立好 2048x2080 的时间线之后，遇到 4096x2160 的素材，create_new_timeline() 函数的 CreatEmptyTimeline() 就不工作了，也就造成 project 的 currentTimeline 还是上一个建立好的时间线。那么函数之后的操作像 current_timeline.SetSetting() 会继续进行，但这次修改的是上一个建立好的时间线，也就是 project 的 currentTimeline。
2. 需要一个机制判断此刻正在新建的时间线是否与之前已经建立好的时间线重复。如果重复，那么直接跳过不再新建时间线。或者 project.SetCurrentTimeline() 到与当前正在建立的时间线重复的那条时间线，这样 current_timeline.SetSetting() 也会被安全跳过，又不会改到上一个建立好的时间线（which is 不是我们想要修改到的时间线）。
3. 怎么判断重复呢：clip_width & clip_height AND timeline_name

In [ ]:
def create_new_timeline(timeline: str, width: str, height: str) -> None:
    """Create new timeline in the _Timeline Bin (The last folder under root folder)"""
    media_pool.SetCurrentFolder(root_folder.GetSubFolderList()[-1])
    timeline_number = project.GetTimelineCount()
    for i in range(timeline_number):
        existing_timeline = project.GetTimelineByIndex(i + 1)
        if existing_timeline.GetName() == timeline or existing_timeline.GetSetting()[
            'timelineResolutionWidth'] == width and existing_timeline.GetSetting()[
            'timelineResolutionHeight'] == height:
            project.SetCurrentTimeline(existing_timeline)

    media_pool.CreateEmptyTimeline(timeline)
    current_timeline = project.GetCurrentTimeline()
    current_timeline.SetSetting("useCustomSettings", "1")
    current_timeline.SetSetting("timelineResolutionWidth", width)
    current_timeline.SetSetting("timelineResolutionHeight", height)
    current_timeline.SetSetting("timelineFrameRate", str(float(25)))

# create_new_timeline() 加入 check ⬆

首先拿到目前有的所有时间线的数量，然后一个 for loop 在这个数量当中，通过 GetTimelineByIndex() 方法一次拿到一条时间线，检查这条时间线的名字是否与函数的输入值 timeline: str 重复，或者这条时间线的长宽与函数的输入值 width: str 和 height: str 重复。如果是的话，我们把已存在的时间线当中的这条与我们准备建立的时间线重复了的时间线，设为 currentTimeline。然后函数接下来的操作将会跳过。也就避免了改到不该改的时间线的情况。

In [ ]:
for clip in sub_folder.GetClipList():
    media_pool.AppendToTimeline(clip)

In [ ]:
from pprint import pprint

import os
from python_get_resolve import GetResolve

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
media_storage = resolve.GetMediaStorage()

media_path = '/Volumes/thom_7_for_Mac/tmp/素材'

sub_folders_full_path = media_storage.GetSubFolderList(media_path)


def absolute_file_paths(directory):
    for dirpath, _, filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))

INVALID_EXTENSION = ["DS_Store", "JPG", "JPEG", "SRT"]

for i in absolute_file_paths('/Volumes/thom_7_for_Mac/tmp/素材'):
    if i.split(".")[-1] not in INVALID_EXTENSION:
        # media_pool.SetCurrentFolder(i.split("/").index("素材") + 1)
        # media_pool.ImportMedia(i)
        print(type(i.split("/")[i.split("/").index("素材") + 1]))